# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [5]:
## SETUP used later

import sys
import os
import json
import pandas as pd
sys.path.insert(1, '/locale/data/jupyter/prando/wd-project/src')
import gt_modules.evaluation as evaluation
from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-directors_g_t-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Directors explorative search") 

Consider the following exploratory information need:

> investigate the results concerning the common aspects between movies directed by Woody Allen or Quentin Tarantino. We want to know the people that worked for both directors with some numerical analyses, what are the differences in terms of budget for their movies, who won more Academy Awards. 

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P106`     | profession   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q2526255`| Director       | node |
| `wd:Q25089`| Woody Allen       | node |
| `wd:Q3772`    | Quentin Tarantino      | node |





Also consider

```
wd:Q25089 ?p ?obj .
```

is the BGP to retrieve all **properties of Woody Allen**


The workload should:


1. Identify the BGP for films

2. Identify the BGP for directors

3. Identify the BGP for workers in a films

4. Compare the workers amongst the films directed by the two directors

5. Return some numerical comparison between the two directors (e.g., how many workers in Tarantino's movies who also worked in Allen's films?, what is the film with the highest number of shared actors? Who is the most used actor by both the directors? etc. )

6. Is the maximum budget for a Tarantino's movie higher of the max budget of an Allen's movie?

7. Who has films with more nominations for Academy Awards and who won more Academy Awards (with his films not only personal awards).

    7.1 Find the BGP for Academy Awards 

    7.2 Find the related subproperties

    7.3 Find how they are related to the directors
    
    7.4 Are there alternative queries to get the same result?



In [6]:
## startup the evaluation
# setup the file and create the empty json
ipname = "directors.ipynb"
pt = os.getcwd()+os.sep+ipname
evaluation.setup(pt)

The index of this workflow is: 5_0


## Task 1 
Identify the BGP for films

In [7]:
#Starting from Woody Allen, look to his properties with a regex to find something like 'director'
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 

?o ?pr wd:Q25089.
?pr <http://schema.org/name> ?name .
FILTER(REGEX(?name,\".*irect.*\"))
} 
LIMIT 100
"""

print("Results")
x = run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P57'), ('name', 'director')]
1


Look for the instance of the objects related with the predicate found in the cell above

In [8]:
queryString = """
SELECT DISTINCT ?ist ?ista
WHERE { 

?o wdt:P57 wd:Q25089;
    wdt:P31 ?ist.
    
?ist <http://schema.org/name> ?ista .
} 
LIMIT 100
"""

print("Results")
x = run_query(queryString)

Results
[('ist', 'http://www.wikidata.org/entity/Q11424'), ('ista', 'film')]
[('ist', 'http://www.wikidata.org/entity/Q506240'), ('ista', 'television film')]
[('ist', 'http://www.wikidata.org/entity/Q5398426'), ('ista', 'television series')]
[('ist', 'http://www.wikidata.org/entity/Q24862'), ('ista', 'short film')]
[('ist', 'http://www.wikidata.org/entity/Q47467768'), ('ista', 'operatic production')]
5


So, Q11424 identifies a film

In [9]:
### insert the result of TASK 1 in the file
film_uri = "http://www.wikidata.org/entity/Q11424"
film_name = "film"
obj = {"uri":film_uri,"name":film_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"1", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 5_0
The path is /locale/data/jupyter/prando/wd-project/2021/ground_truths/gt_json/workflow5_0.json
JSON object updated


## Task 2
Identify the BGP for directors

Since I know that Q11424 is the class of film I will look for the class of the director of a film

In [10]:
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?o wdt:P31 wd:Q11424;
    wdt:P57 ?x.
?x wdt:P31/wdt:P279* ?pr.
?pr <http://schema.org/name> ?name .
FILTER(REGEX(?name,\".*irect.*\"))
} 
LIMIT 100
"""

print("Results")
x= run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/entity/Q2526255'), ('name', 'film director')]
[('pr', 'http://www.wikidata.org/entity/Q3455803'), ('name', 'director')]
2


Q3455803 is a generic director.<br>
Q2526255 is a film director (specifically film e.g., not tv series)

In [12]:
### insert the result of TASK 1 in the file
film_dir_uri = "http://www.wikidata.org/entity/Q2526255"
film_dir_name = "film director"
dir_uri = "http://www.wikidata.org/entity/Q3455803"
dir_name = "director"
obj = [{"uri":film_dir_uri,"name":film_dir_name},{"uri":dir_uri,"name":dir_name}]
evaluation.add_result(evaluation.get_index_workflow(pt),"2", evaluation.TYPE_SINGLE ,"uri", obj ,"any")

The index of this workflow is: 5_0
The path is /locale/data/jupyter/prando/wd-project/2021/ground_truths/gt_json/workflow5_0.json
JSON object updated


## Task 3 

Starting from the class "film director" I look at its properties

In [13]:
queryString = """
SELECT ?p ?pName ?obj ?objName 
WHERE { 

wd:Q2526255 ?p ?obj.

?obj <http://schema.org/name> ?objName .
?p <http://schema.org/name> ?pName .
} 
limit 30
"""

print("Results")
x = run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q15278146'), ('objName', 'Template:Infobox film director')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q222344'), ('objName', 'cinematographer')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1969'), ('objName', 'MovieMeter director ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q3391743'), ('objName', 'visual artist')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q3406858'), ('objName', 'film professional')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/

Found `Q4220920` that represents a filmmaking occupation. Starting from films I will look for human related with such that their profession is an instance of filmmaking occupation or other subclasses.

In [14]:
queryString = """
SELECT DISTINCT ?p ?role
WHERE { 

?film wdt:P31 wd:Q11424 .
?film ?p ?worker .
?worker wdt:P31 wd:Q5 ;
        wdt:P106/wdt:P31/wdt:279* wd:Q4220920 .

?p <http://schema.org/name> ?role .

FILTER(!isLiteral(?worker))
} 
ORDER BY ASC(?role)
"""
#
print("Results")
x = run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P5202'), ('role', 'adapted by')]
[('p', 'http://www.wikidata.org/prop/direct/P1877'), ('role', 'after a work by')]
[('p', 'http://www.wikidata.org/prop/direct/P6942'), ('role', 'animator')]
[('p', 'http://www.wikidata.org/prop/direct/P3174'), ('role', 'art director')]
[('p', 'http://www.wikidata.org/prop/direct/P5126'), ('role', 'assistant director')]
[('p', 'http://www.wikidata.org/prop/direct/P50'), ('role', 'author')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('role', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('role', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P8670'), ('role', 'character designer')]
[('p', 'http://www.wikidata.org/prop/direct/P674'), ('role', 'characters')]
[('p', 'http://www.wikidata.org/prop/direct/P1809'), ('role', 'choreographer')]
[('p', 'http://www.wikidata.org/prop/direct/P86'), ('role', 'composer')]
[('p', 'http://www.wikidata.org/prop/direct/P767'), ('role

In [15]:
obj = [{"uri":r[0][1],"name":r[1][1]} for r in x]
evaluation.add_result(evaluation.get_index_workflow(pt),"3", evaluation.TYPE_SET ,"uri", obj)

The index of this workflow is: 5_0
The path is /locale/data/jupyter/prando/wd-project/2021/ground_truths/gt_json/workflow5_0.json
JSON object updated


## Task 4
Compare the workers amongst the films directed by the two directors

Starting from the BGP above, the predicate director (wdt:P57) and the two director nodes wd:Q25089 (Woody Allen) and wd:Q3772 (Quentin Tarantino) I will perform some queries.

#### Query 1.

**Which are the top 10 films with more people worked on?**

In [16]:
# TOP-10 films with director and people
##values ?work {wd:P57 wd:P161}
queryString = """
SELECT ?film ?director (COUNT(DISTINCT ?worker) AS ?all)
WHERE {
    VALUES ?dir {wd:Q25089 wd:Q3772}
    
    ?o wdt:P31 wd:Q11424 .
    ?o ?p ?worker .
    ?o wdt:P57 ?dir.
    ?worker wdt:P31 wd:Q5 ;
            wdt:P106/wdt:P31/wdt:279* wd:Q4220920 .
    
    ?o sc:name ?film .
    ?dir sc:name ?director .
} 
GROUP BY ?o ?film ?director ?dir
ORDER BY DESC (?all)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'Celebrity'), ('director', 'Woody Allen'), ('all', '60')]
[('film', 'Once Upon a Time in Hollywood'), ('director', 'Quentin Tarantino'), ('all', '60')]
[('film', 'Django Unchained'), ('director', 'Quentin Tarantino'), ('all', '55')]
[('film', 'Inglourious Basterds'), ('director', 'Quentin Tarantino'), ('all', '51')]
[('film', 'New York Stories'), ('director', 'Woody Allen'), ('all', '42')]
[('film', 'The Concert for New York City'), ('director', 'Woody Allen'), ('all', '39')]
[('film', 'Pulp Fiction'), ('director', 'Quentin Tarantino'), ('all', '38')]
[('film', 'Deconstructing Harry'), ('director', 'Woody Allen'), ('all', '38')]
[('film', 'Sin City'), ('director', 'Quentin Tarantino'), ('all', '37')]
[('film', 'Shadows and Fog'), ('director', 'Woody Allen'), ('all', '35')]
10


In [17]:
#list of common workers
queryString = """
SELECT ?worker ?name
WHERE {
    ?film wdt:P31 wd:Q11424 .
    ?film wdt:P57 ?dir.
    ?film ?p ?worker .
    ?worker wdt:P31 wd:Q5 ;
        wdt:P106/wdt:P31/wdt:279* wd:Q4220920 .
    FILTER(?dir = wd:Q3772 || ?dir = wd:Q25089) . 
    FILTER(!isLiteral(?worker)) .
    ?worker sc:name ?name
}
GROUP BY ?worker ?name
HAVING (COUNT(DISTINCT ?dir) = 2)
"""

print("Results")
x = run_query(queryString)

Results
[('worker', 'http://www.wikidata.org/entity/Q346285'), ('name', 'David Arnold')]
[('worker', 'http://www.wikidata.org/entity/Q185724'), ('name', 'Mike Myers')]
[('worker', 'http://www.wikidata.org/entity/Q207596'), ('name', 'Daryl Hannah')]
[('worker', 'http://www.wikidata.org/entity/Q202148'), ('name', 'Burt Reynolds')]
[('worker', 'http://www.wikidata.org/entity/Q231096'), ('name', 'Léa Seydoux')]
[('worker', 'http://www.wikidata.org/entity/Q38111'), ('name', 'Leonardo DiCaprio')]
[('worker', 'http://www.wikidata.org/entity/Q185051'), ('name', 'Christopher Walken')]
[('worker', 'http://www.wikidata.org/entity/Q109232'), ('name', 'Rosanna Arquette')]
[('worker', 'http://www.wikidata.org/entity/Q203804'), ('name', 'Tim Roth')]
[('worker', 'http://www.wikidata.org/entity/Q36949'), ('name', 'Robert De Niro')]
[('worker', 'http://www.wikidata.org/entity/Q76819'), ('name', 'Christoph Waltz')]
[('worker', 'http://www.wikidata.org/entity/Q125017'), ('name', 'Uma Thurman')]
[('worker'

## Task 5
Return some numerical comparisons between the two directors (e.g., how many workers in Tarantino's movies who also worked in Allen's films? what is the film with the highest number of shared actors? Who is the most used actor by both the directors? etc. )

**How many workers in Tarantino's movies who also worked in Allen's films?**

In [20]:
#how many common workers?
queryString = """
SELECT COUNT(*) AS ?common_workers WHERE{
    SELECT ?worker ?name
    WHERE {
        VALUES ?dir {wd:Q3772 wd:Q25089}
        ?film wdt:P31 wd:Q11424 .
        ?film wdt:P57 ?dir.
        ?film ?p ?worker .
        ?worker wdt:P31 wd:Q5 ;
            wdt:P106/wdt:P31/wdt:279* wd:Q4220920 .
        FILTER(!isLiteral(?worker)) .
        ?worker sc:name ?name
    }
    GROUP BY ?worker ?name
    HAVING (COUNT(DISTINCT ?dir) = 2)
}
"""

print("Results")
x=run_query(queryString)

Results
[('common_workers', '14')]
1


**What is the film with the highest number of shared actors?**

First: find the actors who worked for both directors

In [22]:
#list of shared actors
queryString = """
SELECT DISTINCT ?name AS ?shared
WHERE { 

    # allen
    ?filmA wdt:P31 wd:Q11424 .
    ?filmA wdt:P57 wd:Q25089.
    ?filmA wdt:P161 ?actors .
    
    # tarantino
    ?filmT wdt:P31 wd:Q11424 .
    ?filmT wdt:P57 wd:Q3772 .
    ?filmT wdt:P161 ?actors .
    
    ?actors sc:name ?name.
} 
"""

print("Results")
x=run_query(queryString)

Results
[('shared', 'Mike Myers')]
[('shared', 'Tim Roth')]
[('shared', 'Steve Buscemi')]
[('shared', 'Léa Seydoux')]
[('shared', 'Rosanna Arquette')]
[('shared', 'Daryl Hannah')]
[('shared', 'Robert De Niro')]
[('shared', 'Uma Thurman')]
[('shared', 'Christopher Walken')]
[('shared', 'Leonardo DiCaprio')]
[('shared', 'Burt Reynolds')]
[('shared', 'Christoph Waltz')]
12


In [23]:
#films with more shared actors
queryString = """
SELECT DISTINCT ?f ?film (COUNT(DISTINCT ?actor) AS ?more)
WHERE{
    {
        SELECT DISTINCT ?actors AS ?shared
        WHERE { 

            # allen
            ?filmA wdt:P31 wd:Q11424 .
            ?filmA wdt:P57 wd:Q25089.
            ?filmA wdt:P161 ?actors .

            # tarantino
            ?filmT wdt:P31 wd:Q11424 .
            ?filmT wdt:P57 wd:Q3772 .
            ?filmT wdt:P161 ?actors .

        } 
    }
    ?f wdt:P31 wd:Q11424;
        wdt:P161 ?x.
    ?f sc:name ?film.
    ?shared sc:name ?actor.
    FILTER(?x = ?shared)
}

GROUP BY ?f ?film
ORDER BY DESC (?more)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('f', 'http://www.wikidata.org/entity/Q104123'), ('film', 'Pulp Fiction'), ('more', '5')]
[('f', 'http://www.wikidata.org/entity/Q3986392'), ('film', 'The Concert for New York City'), ('more', '4')]
[('f', 'http://www.wikidata.org/entity/Q1547511'), ('film', 'One Hundred and One Nights'), ('more', '3')]
[('f', 'http://www.wikidata.org/entity/Q153723'), ('film', 'Inglourious Basterds'), ('more', '3')]
[('f', 'http://www.wikidata.org/entity/Q47300912'), ('film', 'Once Upon a Time in Hollywood'), ('more', '3')]
[('f', 'http://www.wikidata.org/entity/Q25999873'), ('film', 'The War with Grandpa'), ('more', '3')]
[('f', 'http://www.wikidata.org/entity/Q1163526'), ('film', 'Search and Destroy'), ('more', '2')]
[('f', 'http://www.wikidata.org/entity/Q1743746'), ('film', 'Kiss Daddy Goodnight'), ('more', '2')]
[('f', 'http://www.wikidata.org/entity/Q775586'), ('film', 'King of New York'), ('more', '2')]
[('f', 'http://www.wikidata.org/entity/Q1524544'), ('film', 'Deceiver'), ('more', '

In [25]:
#films with more shared actors with their names
queryString = """
SELECT DISTINCT ?film ?director (COUNT(DISTINCT ?actor) AS ?more) (GROUP_CONCAT(?actor;separator=",") AS ?actors)
WHERE{
    {
        SELECT DISTINCT ?actors AS ?shared
        WHERE { 

            # allen
            ?filmA wdt:P31 wd:Q11424 .
            ?filmA wdt:P57 wd:Q25089.
            ?filmA wdt:P161 ?actors .

            # tarantino
            ?filmT wdt:P31 wd:Q11424 .
            ?filmT wdt:P57 wd:Q3772 .
            ?filmT wdt:P161 ?actors .

        } 
    }
    ?f wdt:P31 wd:Q11424;
        wdt:P57 ?dir;
        wdt:P161 ?x.
    ?f sc:name ?film.
    ?dir sc:name ?director.
    ?shared sc:name ?actor.
    FILTER(?x = ?shared)
}

GROUP BY ?f ?director ?film
ORDER BY DESC (?more)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'Pulp Fiction'), ('director', 'Quentin Tarantino'), ('more', '5'), ('actors', 'Tim Roth,Steve Buscemi,Rosanna Arquette,Uma Thurman,Christopher Walken')]
[('film', 'The Concert for New York City'), ('director', 'Spike Lee'), ('more', '4'), ('actors', 'Mike Myers,Steve Buscemi,Robert De Niro,Leonardo DiCaprio')]
[('film', 'The Concert for New York City'), ('director', 'Martin Scorsese'), ('more', '4'), ('actors', 'Mike Myers,Steve Buscemi,Robert De Niro,Leonardo DiCaprio')]
[('film', 'The Concert for New York City'), ('director', 'Kevin Smith'), ('more', '4'), ('actors', 'Mike Myers,Steve Buscemi,Robert De Niro,Leonardo DiCaprio')]
[('film', 'The Concert for New York City'), ('director', 'Jerry Seinfeld'), ('more', '4'), ('actors', 'Mike Myers,Steve Buscemi,Robert De Niro,Leonardo DiCaprio')]
[('film', 'The Concert for New York City'), ('director', 'Woody Allen'), ('more', '4'), ('actors', 'Mike Myers,Steve Buscemi,Robert De Niro,Leonardo DiCaprio')]
[('film', 'The Conc

**Who is the most used actor by both the directors?**

In [26]:
queryString = """
SELECT DISTINCT ?director ?actor (COUNT(DISTINCT ?f) AS ?films)
WHERE{
    {
        SELECT DISTINCT ?actors AS ?shared
        WHERE { 

            # allen
            ?filmA wdt:P31 wd:Q11424 .
            ?filmA wdt:P57 wd:Q25089.
            ?filmA wdt:P161 ?actors .

            # tarantino
            ?filmT wdt:P31 wd:Q11424 .
            ?filmT wdt:P57 wd:Q3772 .
            ?filmT wdt:P161 ?actors .

        } 
    }
    ?f wdt:P31 wd:Q11424;
        wdt:P57 ?dir;
        wdt:P161 ?x.
    ?f sc:name ?film.
    ?dir sc:name ?director.
    ?shared sc:name ?actor.
    FILTER((?dir = wd:Q25089 || ?dir = wd:Q3772) && (?x = ?shared))
}

GROUP BY ?common ?actor ?dir ?director
ORDER BY DESC (?films)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('director', 'Quentin Tarantino'), ('actor', 'Uma Thurman'), ('films', '4')]
[('director', 'Quentin Tarantino'), ('actor', 'Tim Roth'), ('films', '4')]
[('director', 'Quentin Tarantino'), ('actor', 'Daryl Hannah'), ('films', '3')]
[('director', 'Quentin Tarantino'), ('actor', 'Christoph Waltz'), ('films', '2')]
[('director', 'Woody Allen'), ('actor', 'Leonardo DiCaprio'), ('films', '2')]
[('director', 'Quentin Tarantino'), ('actor', 'Steve Buscemi'), ('films', '2')]
[('director', 'Quentin Tarantino'), ('actor', 'Leonardo DiCaprio'), ('films', '2')]
[('director', 'Woody Allen'), ('actor', 'Steve Buscemi'), ('films', '2')]
[('director', 'Woody Allen'), ('actor', 'Burt Reynolds'), ('films', '1')]
[('director', 'Quentin Tarantino'), ('actor', 'Robert De Niro'), ('films', '1')]
10


In [27]:
queryString = """
SELECT DISTINCT ?actor (COUNT(DISTINCT ?f) AS ?films)
WHERE{
    {
        SELECT DISTINCT ?actors AS ?shared
        WHERE { 

            # allen
            ?filmA wdt:P31 wd:Q11424 .
            ?filmA wdt:P57 wd:Q25089.
            ?filmA wdt:P161 ?actors .

            # tarantino
            ?filmT wdt:P31 wd:Q11424 .
            ?filmT wdt:P57 wd:Q3772 .
            ?filmT wdt:P161 ?actors .

        } 
    }
    ?f wdt:P31 wd:Q11424;
        wdt:P57 ?dir;
        wdt:P161 ?x.
    ?f sc:name ?film.
    ?dir sc:name ?director.
    ?shared sc:name ?actor.
    FILTER((?dir = wd:Q25089 || ?dir = wd:Q3772) && (?x = ?shared))
}

GROUP BY ?common ?actor
ORDER BY DESC (?films)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('actor', 'Uma Thurman'), ('films', '5')]
[('actor', 'Tim Roth'), ('films', '5')]
[('actor', 'Leonardo DiCaprio'), ('films', '4')]
[('actor', 'Daryl Hannah'), ('films', '4')]
[('actor', 'Steve Buscemi'), ('films', '4')]
[('actor', 'Christoph Waltz'), ('films', '3')]
[('actor', 'Léa Seydoux'), ('films', '2')]
[('actor', 'Robert De Niro'), ('films', '2')]
[('actor', 'Mike Myers'), ('films', '2')]
[('actor', 'Rosanna Arquette'), ('films', '2')]
10


## Task 6
Is the maximum budget for a Tarantino's movie higher of the max budget of an Allen's movie?

In [18]:
# look for the budget
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?film wdt:P31 wd:Q11424;
     wdt:P57 wd:Q25089;
    ?pr ?x.
?pr <http://schema.org/name> ?name .
FILTER(isNumeric(?x))
} 
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P2047'), ('name', 'duration')]
[('pr', 'http://www.wikidata.org/prop/direct/P2130'), ('name', 'cost')]
[('pr', 'http://www.wikidata.org/prop/direct/P2142'), ('name', 'box office')]
[('pr', 'http://www.wikidata.org/prop/direct/P2769'), ('name', 'budget')]
4


Find two properties: cost (`wd:P2130`) and budget (`wd:P2769`). Have a look to both

In [19]:
#result with only numbers using property budget (P2769)
queryString = """
SELECT ?dir MAX( ?cost ) AS ?costT
WHERE {
    ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?dir ;
        wdt:P2769 ?cost .
    FILTER(?dir = wd:Q25089 || ?dir = wd:Q3772 ).
}
GROUP BY ?dir
"""

print("Results")
x=run_query(queryString)

Results
[('dir', 'http://www.wikidata.org/entity/Q25089'), ('costT', '25000000')]
1


In [20]:
#result with only numbers using property cost (P2130)
queryString = """
SELECT ?dir MAX( ?cost ) AS ?costT
WHERE {
    ?film wdt:P31 wd:Q11424 ;
        wdt:P57 ?dir ;
        wdt:P2130 ?cost .
    FILTER(?dir = wd:Q25089 || ?dir = wd:Q3772 ).
}
GROUP BY ?dir
"""

print("Results")
x=run_query(queryString)

Results
[('dir', 'http://www.wikidata.org/entity/Q3772'), ('costT', '100000000')]
[('dir', 'http://www.wikidata.org/entity/Q25089'), ('costT', '22000000')]
2


In [21]:
#result with film name
queryString = """
SELECT ?dir ?dir_name ?film_name (?c AS ?max_cost)
WHERE{
{
    SELECT ?dir MAX( ?cost ) AS ?costT
    WHERE {
        ?film wdt:P31 wd:Q11424 ;
            wdt:P57 ?dir ;
            wdt:P2130 ?cost .
        FILTER(?dir = wd:Q25089 || ?dir = wd:Q3772 ).
    }GROUP BY ?dir
}
?f wdt:P31 wd:Q11424 ;
    wdt:P57 ?d ;
    wdt:P2130 ?c .
FILTER(?d = ?dir && ?c = ?costT).
?f sc:name ?film_name.
?d sc:name ?dir_name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('dir', 'http://www.wikidata.org/entity/Q25089'), ('dir_name', 'Woody Allen'), ('film_name', 'You Will Meet a Tall Dark Stranger'), ('max_cost', '22000000')]
[('dir', 'http://www.wikidata.org/entity/Q3772'), ('dir_name', 'Quentin Tarantino'), ('film_name', 'Django Unchained'), ('max_cost', '100000000')]
2


Found that cost is the property to answer the question. Run an ASK query

In [25]:
### insert the result of TASK 6 in the file
obj1 = {"uri":x[0][0][1],"name":x[0][1][1],"value":x[0][3][1]}
obj2 = {"uri":x[1][0][1],"name":x[1][1][1],"value":x[1][3][1]}
obj = {"ask":True,"elements":[obj1,obj2],"check":"value","format":"number"}
evaluation.add_result(evaluation.get_index_workflow(pt),"6", evaluation.TYPE_SINGLE ,"ask",[obj],type_inside=evaluation.TYPE_SINGLE)

The index of this workflow is: 5_0
The path is /locale/data/jupyter/prando/wd-project/2021/ground_truths/gt_json/workflow5_0.json
JSON object updated


In [26]:
queryString = """
ASK WHERE { 
{
#Tarantino
    SELECT MAX( ?cost ) AS ?costT
    WHERE {
        ?film wdt:P31 wd:Q11424 ;
            wdt:P57 wd:Q3772 ;
            wdt:P2130 ?cost .
    }
}
{
#Allen
    SELECT MAX(?cost) AS ?costA
    WHERE {
        ?film wdt:P31 wd:Q11424 ;
            wdt:P57 wd:Q25089 ;
            wdt:P2130 ?cost .
    }
}

FILTER(?costT > ?costA)

} 
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

Yes because the maximum budget of Tarantino is 100M while the maximum budget of Allen is 22M. 

## Task 7 
Who has films with more nominations for Academy Awards and who won more Academy Awards (with his films not only personal awards).

### Task 7.1
Find the BGP for Academy Awards

Start from a film looking for property that are similar to `nominated`

In [25]:
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?f wdt:P31 wd:Q11424;
    ?pr ?a.
?pr sc:name ?name .
FILTER(REGEX(?name,".*nominated.*"))
} 
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('pr', 'http://www.wikidata.org/prop/direct/P4353'), ('name', 'nominated by')]
2


In [26]:
#looking for what are the instances of these objects
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
?f wdt:P31 wd:Q11424;
    wdt:P1411 ?a.
?a wdt:P31 ?x.
?x sc:name ?name .
FILTER(REGEX(?name,".*award.*"))
} 
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q96474676'), ('name', 'award for best leading actress')]
[('x', 'http://www.wikidata.org/entity/Q96474688'), ('name', 'award for best film by genre')]
[('x', 'http://www.wikidata.org/entity/Q96474699'), ('name', 'award for best costume design')]
[('x', 'http://www.wikidata.org/entity/Q96474679'), ('name', 'award for best direction')]
[('x', 'http://www.wikidata.org/entity/Q618779'), ('name', 'award')]
[('x', 'http://www.wikidata.org/entity/Q96474603'), ('name', 'award for best supporting actor')]
[('x', 'http://www.wikidata.org/entity/Q56116950'), ('name', 'film award category')]
[('x', 'http://www.wikidata.org/entity/Q96474683'), ('name', 'award for best sound editing')]
[('x', 'http://www.wikidata.org/entity/Q96474709'), ('name', 'award for best visual effects')]
[('x', 'http://www.wikidata.org/entity/Q4220917'), ('name', 'film award')]
[('x', 'http://www.wikidata.org/entity/Q96474687'), ('name', 'award for best film')]
[('x', 'http://ww

`wd:Q4220917`(film award) seems to be the right class of award. Try to look for subclasses of this class to find the Academy award

In [27]:
queryString = """
SELECT DISTINCT ?a ?name
WHERE { 
?a wdt:P279+ wd:Q4220917.
?a sc:name ?name .
FILTER(REGEX(?name,".*cadem.*"))
} 
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q655089'), ('name', 'International Indian Film Academy Awards')]
[('a', 'http://www.wikidata.org/entity/Q2510955'), ('name', 'Japan Academy Prize for Animation of the Year')]
[('a', 'http://www.wikidata.org/entity/Q19020'), ('name', 'Academy Awards')]
[('a', 'http://www.wikidata.org/entity/Q194258'), ('name', 'Japan Academy Film Prize')]
[('a', 'http://www.wikidata.org/entity/Q732997'), ('name', 'British Academy Film Awards')]
[('a', 'http://www.wikidata.org/entity/Q384139'), ('name', 'Africa Movie Academy Award')]
[('a', 'http://www.wikidata.org/entity/Q11024975'), ('name', 'Myanmar Motion Picture Academy Awards')]
[('a', 'http://www.wikidata.org/entity/Q18345192'), ('name', 'Golden Icons Academy Movie Awards')]
[('a', 'http://www.wikidata.org/entity/Q74066919'), ('name', '14th Africa Movie Academy Awards')]
9


`wd:Q19020` is the Academy Awards

In [27]:
### insert the result of TASK 7.1 in the file
ac_aw_uri = "http://www.wikidata.org/entity/Q19020"
ac_aw_name = "Academy Awards"
obj = {"uri":ac_aw_uri,"name":ac_aw_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"7.1", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 5_0
The path is /locale/data/jupyter/prando/wd-project/2021/ground_truths/gt_json/workflow5_0.json
JSON object updated


### Task 7.2 
Find the related subproperties

In [28]:
#look for something istance of academy awards
queryString = """
SELECT DISTINCT ?a ?name
WHERE { 
?a wdt:P31 wd:Q19020.
?a sc:name ?name .
} 
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q102427'), ('name', 'Academy Award for Best Picture')]
[('a', 'http://www.wikidata.org/entity/Q103360'), ('name', 'Academy Award for Best Director')]
[('a', 'http://www.wikidata.org/entity/Q103618'), ('name', 'Academy Award for Best Actress')]
[('a', 'http://www.wikidata.org/entity/Q106301'), ('name', 'Academy Award for Best Supporting Actress')]
[('a', 'http://www.wikidata.org/entity/Q107258'), ('name', 'Academy Award for Best Writing, Adapted Screenplay')]
[('a', 'http://www.wikidata.org/entity/Q112243'), ('name', 'Academy Award for Best Original Song')]
[('a', 'http://www.wikidata.org/entity/Q281939'), ('name', 'Academy Award for Best Film Editing')]
[('a', 'http://www.wikidata.org/entity/Q41417'), ('name', 'Academy Award for Best Writing, Original Screenplay')]
[('a', 'http://www.wikidata.org/entity/Q844273'), ('name', 'Irving G. Thalberg Memorial Award')]
[('a', 'http://www.wikidata.org/entity/Q106800'), ('name', 'Academy Award for Be

`wd:Q19020` is the class of Academy Awards. Then selecting objects that are istance of academy awards we have the list of all type of academy awards. 

In [29]:
obj = [{"uri":r[0][1],"name":r[1][1]} for r in x]
evaluation.add_result(evaluation.get_index_workflow(pt),"4", evaluation.TYPE_SET ,"uri", obj)

The index of this workflow is: 5_0
The path is /locale/data/jupyter/prando/wd-project/2021/ground_truths/gt_json/workflow5_0.json
JSON object updated


In [31]:
#look for some properties
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?a wdt:P31 wd:Q19020.
?a ?pr ?o.
?pr sc:name ?name .
} 
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('pr', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('pr', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('pr', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('pr', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('pr', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('pr', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('pr', 'http://www.wikidata.org/prop/direct/P1705'), ('name', 'native label')]
[('pr', 'http://www.wikidata.org/prop/direct/P2257'), ('name', 'event interval')]
[('pr', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('pr

In [32]:
#look for some properties
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?a wdt:P31 wd:Q19020.
?o ?pr ?a.
?pr sc:name ?name .
} 
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('pr', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('pr', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('pr', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('pr', 'http://www.wikidata.org/prop/direct/P2522'), ('name', 'victory')]
[('pr', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('pr', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('pr', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('pr', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('pr', 'http://www.wikidata.org/prop/direct/P921'), ('name', 'main subject')]
[('pr', 'http://www.wikidata.org/prop/direct/P971'), ('name', 'category combines topics')]
12


The properties of interest are:
- `wdt:P1411`  nominated for
- `wdt:P166`  award received

### Task 7.3
Find how they are related to the directors (`wd:Q3455803`)

In [33]:
#find all people who are connected with the class director
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?sub wdt:P279* wd:Q3455803.
wd:Q3772 ?pr ?sub.

?pr sc:name ?name .
} 
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
1


In [30]:
#from the directors find the relations to the academy award
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
?award wdt:P31 wd:Q19020.
?x wdt:P106 wd:Q3455803.

?x ?pr ?award.

?pr sc:name ?name .
} 
LIMIT 100
"""

print("Results")
x = run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('pr', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
2


In [31]:
obj = [{"uri":r[0][1],"name":r[1][1]} for r in x]
evaluation.add_result(evaluation.get_index_workflow(pt),"7.3", evaluation.TYPE_SET ,"uri", obj)

The index of this workflow is: 5_0
The path is /locale/data/jupyter/prando/wd-project/2021/ground_truths/gt_json/workflow5_0.json
JSON object updated


### Task 7 Who has more nomination and more wins? ONE SHOT VERSION

In [33]:
#academy award nominations and wins by directors
queryString = """
SELECT DISTINCT ?dir_wins ?director ?nominations ?wins
WHERE { 
{
    SELECT DISTINCT ?dir AS ?dir_nom (COUNT(*) AS ?nominations)
    WHERE { 
    ?film wdt:P31 wd:Q11424 .
    ?film wdt:P57 ?dir.
    ?film wdt:P1411 ?nomination .
    ?nomination wdt:P31 wd:Q19020 .
    FILTER(?dir = wd:Q25089 || ?dir = wd:Q3772 ).
    } 
    GROUP BY ?dir 
}
{
    SELECT DISTINCT ?dir AS ?dir_wins (COUNT(*) AS ?wins)
    WHERE { 
    ?film wdt:P31 wd:Q11424 .
    ?film wdt:P57 ?dir.
    ?film wdt:P166 ?win .
    ?win wdt:P31 wd:Q19020 .
    FILTER(?dir = wd:Q25089 || ?dir = wd:Q3772 ).
    } 
    GROUP BY ?dir
}
FILTER(?dir_wins=?dir_nom)
?dir_wins sc:name ?director
} 
ORDER BY DESC ( ?nominations) DESC (?wins)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('dir_wins', 'http://www.wikidata.org/entity/Q25089'), ('director', 'Woody Allen'), ('nominations', '52'), ('wins', '12')]
[('dir_wins', 'http://www.wikidata.org/entity/Q3772'), ('director', 'Quentin Tarantino'), ('nominations', '24'), ('wins', '7')]
2


In [34]:
## more literals associated to the same element
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    nom = i[2][1]
    win = i[3][1]
    for val in [nom,win]:
        obj = {}
        obj["refers_to"] = f_uri
        obj["refers_to_name"] = f_name
        obj["check"] = "value"
        obj["value"]= val
        objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"7", evaluation.TYPE_REFERRED ,"value", objs, elements_per_tuple = 2)

The index of this workflow is: 5_0
The path is /locale/data/jupyter/prando/wd-project/2021/ground_truths/gt_json/workflow5_0.json
JSON object updated


### Task 7 Who has more nomination and more wins? EXTENDED VERSION

**Academy awards di Woody Allen - Nomination**

In [79]:
queryString = """
SELECT DISTINCT ?filmName ?nomination ?nomName 
WHERE { 
    ?film wdt:P31 wd:Q11424 .
    ?film wdt:P57 wd:Q25089.
    ?film wdt:P1411 ?nomination .
    ?nomination wdt:P31 wd:Q19020 .
    
    ?film <http://schema.org/name> ?filmName .
    ?nomination <http://schema.org/name> ?nomName .
} 
ORDER BY ASC(?filmName)
"""

print("Results")
x=run_query(queryString)

Results
[('filmName', 'Alice'), ('nomination', 'http://www.wikidata.org/entity/Q41417'), ('nomName', 'Academy Award for Best Writing, Original Screenplay')]
[('filmName', 'Annie Hall'), ('nomination', 'http://www.wikidata.org/entity/Q102427'), ('nomName', 'Academy Award for Best Picture')]
[('filmName', 'Annie Hall'), ('nomination', 'http://www.wikidata.org/entity/Q103360'), ('nomName', 'Academy Award for Best Director')]
[('filmName', 'Annie Hall'), ('nomination', 'http://www.wikidata.org/entity/Q103618'), ('nomName', 'Academy Award for Best Actress')]
[('filmName', 'Annie Hall'), ('nomination', 'http://www.wikidata.org/entity/Q41417'), ('nomName', 'Academy Award for Best Writing, Original Screenplay')]
[('filmName', 'Annie Hall'), ('nomination', 'http://www.wikidata.org/entity/Q103916'), ('nomName', 'Academy Award for Best Actor')]
[('filmName', 'Blue Jasmine'), ('nomination', 'http://www.wikidata.org/entity/Q103618'), ('nomName', 'Academy Award for Best Actress')]
[('filmName', 'Blu

**Academy awards di Woody Allen - Wins**

In [80]:
queryString = """
SELECT DISTINCT ?filmName ?win ?winName
WHERE { 
    ?film wdt:P31 wd:Q11424 .
    ?film wdt:P57 wd:Q25089.
    ?film wdt:P166 ?win .
    ?win wdt:P31 wd:Q19020 .
    
    ?film <http://schema.org/name> ?filmName .
    ?win <http://schema.org/name> ?winName .
} 
ORDER BY ASC(?filmName)
"""

print("Results")
x=run_query(queryString)

Results
[('filmName', 'Annie Hall'), ('win', 'http://www.wikidata.org/entity/Q103360'), ('winName', 'Academy Award for Best Director')]
[('filmName', 'Annie Hall'), ('win', 'http://www.wikidata.org/entity/Q41417'), ('winName', 'Academy Award for Best Writing, Original Screenplay')]
[('filmName', 'Annie Hall'), ('win', 'http://www.wikidata.org/entity/Q102427'), ('winName', 'Academy Award for Best Picture')]
[('filmName', 'Annie Hall'), ('win', 'http://www.wikidata.org/entity/Q103618'), ('winName', 'Academy Award for Best Actress')]
[('filmName', 'Blue Jasmine'), ('win', 'http://www.wikidata.org/entity/Q103618'), ('winName', 'Academy Award for Best Actress')]
[('filmName', 'Bullets Over Broadway'), ('win', 'http://www.wikidata.org/entity/Q106301'), ('winName', 'Academy Award for Best Supporting Actress')]
[('filmName', 'Hannah and Her Sisters'), ('win', 'http://www.wikidata.org/entity/Q41417'), ('winName', 'Academy Award for Best Writing, Original Screenplay')]
[('filmName', 'Hannah and 

**Academy awards di Quentin Tarantino - Nomination**

In [81]:
queryString = """
SELECT DISTINCT ?filmName ?nomination ?nomName
WHERE { 
    ?film wdt:P31 wd:Q11424 .
    ?film wdt:P57 wd:Q3772.
    ?film wdt:P1411 ?nomination .
    ?nomination wdt:P31 wd:Q19020 .
    
    ?film <http://schema.org/name> ?filmName .
    ?nomination <http://schema.org/name> ?nomName .
} 
ORDER BY ASC(?filmName)
"""

print("Results")
x=run_query(queryString)

Results
[('filmName', 'Django Unchained'), ('nomination', 'http://www.wikidata.org/entity/Q106291'), ('nomName', 'Academy Award for Best Supporting Actor')]
[('filmName', 'Django Unchained'), ('nomination', 'http://www.wikidata.org/entity/Q102427'), ('nomName', 'Academy Award for Best Picture')]
[('filmName', 'Django Unchained'), ('nomination', 'http://www.wikidata.org/entity/Q131520'), ('nomName', 'Academy Award for Best Cinematography')]
[('filmName', 'Django Unchained'), ('nomination', 'http://www.wikidata.org/entity/Q488645'), ('nomName', 'Academy Award for Best Sound Editing')]
[('filmName', 'Django Unchained'), ('nomination', 'http://www.wikidata.org/entity/Q41417'), ('nomName', 'Academy Award for Best Writing, Original Screenplay')]
[('filmName', 'Inglourious Basterds'), ('nomination', 'http://www.wikidata.org/entity/Q106291'), ('nomName', 'Academy Award for Best Supporting Actor')]
[('filmName', 'Inglourious Basterds'), ('nomination', 'http://www.wikidata.org/entity/Q102427'), 

**Academy awards di Quentin Tarantino - Wins**

In [82]:
queryString = """
SELECT DISTINCT ?filmName ?win ?winName
WHERE { 
    ?film wdt:P31 wd:Q11424 .
    ?film wdt:P57 wd:Q3772.
    ?film wdt:P166 ?win .
    ?win wdt:P31 wd:Q19020 .
    
    ?film <http://schema.org/name> ?filmName .
    ?win <http://schema.org/name> ?winName .
} 
ORDER BY ASC(?filmName)
"""

print("Results")
x=run_query(queryString)

Results
[('filmName', 'Django Unchained'), ('win', 'http://www.wikidata.org/entity/Q106291'), ('winName', 'Academy Award for Best Supporting Actor')]
[('filmName', 'Django Unchained'), ('win', 'http://www.wikidata.org/entity/Q41417'), ('winName', 'Academy Award for Best Writing, Original Screenplay')]
[('filmName', 'Inglourious Basterds'), ('win', 'http://www.wikidata.org/entity/Q106291'), ('winName', 'Academy Award for Best Supporting Actor')]
[('filmName', 'Once Upon a Time in Hollywood'), ('win', 'http://www.wikidata.org/entity/Q277751'), ('winName', 'Academy Award for Best Production Design')]
[('filmName', 'Once Upon a Time in Hollywood'), ('win', 'http://www.wikidata.org/entity/Q106291'), ('winName', 'Academy Award for Best Supporting Actor')]
[('filmName', 'Pulp Fiction'), ('win', 'http://www.wikidata.org/entity/Q41417'), ('winName', 'Academy Award for Best Writing, Original Screenplay')]
[('filmName', 'The Hateful Eight'), ('win', 'http://www.wikidata.org/entity/Q488651'), ('wi

**Woody Allen has more nomination than Quentin Tarantino? YES**

In [83]:
queryString = """
ASK WHERE
{
{
    SELECT COUNT(?nomination) AS ?wa_nom 
    WHERE { 
        ?film wdt:P31 wd:Q11424 .
        ?film wdt:P57 wd:Q25089.
        ?film wdt:P1411 ?nomination .
        ?nomination wdt:P31 wd:Q19020 .
    }
}
{
    SELECT COUNT(?nomination) AS ?qt_nom 
    WHERE { 
        ?film wdt:P31 wd:Q11424 .
        ?film wdt:P57 wd:Q3772.
        ?film wdt:P1411 ?nomination .
        ?nomination wdt:P31 wd:Q19020 .
    }
}
FILTER(?wa_nom > ?qt_nom)
}
"""

print("Results")
x=run_ask_query(queryString)

Results


**Woody Allen has more wins than Quentin Tarantino? YES**

In [85]:
queryString = """
ASK WHERE
{
{
    SELECT COUNT(?nomination) AS ?wa_win 
    WHERE { 
        ?film wdt:P31 wd:Q11424 .
        ?film wdt:P57 wd:Q25089.
        ?film wdt:P166 ?nomination .
        ?nomination wdt:P31 wd:Q19020 .
    }
}
{
    SELECT COUNT(?nomination) AS ?qt_win
    WHERE { 
        ?film wdt:P31 wd:Q11424 .
        ?film wdt:P57 wd:Q3772.
        ?film wdt:P166 ?nomination .
        ?nomination wdt:P31 wd:Q19020 .
    }
}
FILTER(?wa_win > ?qt_win)
}
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}